In [ ]:
import cv2
import mediapipe as mp
import time
import os
import random
import urllib.request
import matplotlib.pyplot as plt # <--- NEW: For the Graphs
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# --- SETUP ---
model_filename = 'face_landmarker.task'
if not os.path.exists(model_filename):
    url = "https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task"
    urllib.request.urlretrieve(url, model_filename)

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_filename),
    running_mode=VisionRunningMode.VIDEO,
    num_faces=1
)

def get_gaze_ratio(pupil, inner, outer, width):
    p_x = pupil.x * width
    i_x = inner.x * width
    o_x = outer.x * width
    dist_inner = abs(p_x - i_x)
    dist_outer = abs(p_x - o_x)
    if dist_inner == 0: dist_inner = 0.001
    return dist_outer / dist_inner

# --- VARIABLES ---
state = "WAIT"
target_pos = (0, 0)
target_side = "NONE"
reaction_start_time = 0
last_result_ms = 0

# NEW: RESEARCH DATA STORAGE
trial_data = []      # Stores the history of every jump
max_velocity = 0     # Tracks top speed during a jump
prev_ratio = 1.0     # Needed to calculate speed (change in position)

cap = cv2.VideoCapture(0)

# Full Screen
window_name = "Oculomotor Experiment"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

with FaceLandmarker.create_from_options(options) as landmarker:
    print("✅ STARTING. Do 10 jumps, then press ESC.")
    start_time = time.time()

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        timestamp_ms = int((time.time() - start_time) * 1000)
        
        detection_result = landmarker.detect_for_video(mp_image, timestamp_ms)
        h, w, _ = frame.shape

        if detection_result.face_landmarks:
            landmarks = detection_result.face_landmarks[0]
            pupil = landmarks[473]
            inner = landmarks[362]
            outer = landmarks[263]
            current_ratio = get_gaze_ratio(pupil, inner, outer, w)
            
            # --- NEW: VELOCITY CALCULATION ---
            # Speed = How much the ratio changed since last frame
            instant_velocity = abs(current_ratio - prev_ratio) * 100 
            
            # If we are in the middle of a test, track the Peak Speed
            if state == "TARGET":
                if instant_velocity > max_velocity:
                    max_velocity = instant_velocity
            
            prev_ratio = current_ratio

            # Visuals
            cx, cy = int(pupil.x * w), int(pupil.y * h)
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)

            # --- GAME LOGIC ---
            if state == "WAIT":
                cv2.line(frame, (w//2 - 20, h//2), (w//2 + 20, h//2), (200, 200, 200), 2)
                cv2.line(frame, (w//2, h//2 - 20), (w//2, h//2 + 20), (200, 200, 200), 2)
                cv2.putText(frame, f"TRIALS: {len(trial_data)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                
                # YOUR CALIBRATION (0.9 to 1.1)
                if 0.9 < current_ratio < 1.1:
                    if random.randint(1, 100) < 5:
                        state = "TARGET"
                        max_velocity = 0 # Reset speed tracker
                        if random.choice([True, False]):
                            target_side = "LEFT"
                            target_pos = (100, h//2)
                        else:
                            target_side = "RIGHT"
                            target_pos = (w-100, h//2)
                        reaction_start_time = time.time()

            elif state == "TARGET":
                cv2.circle(frame, target_pos, 25, (0, 255, 0), -1)
                
                success_move = False
                # YOUR CALIBRATION (Look Left/Right)
                if target_side == "LEFT" and current_ratio > 1.2: 
                    success_move = True
                elif target_side == "RIGHT" and current_ratio < 0.8: 
                    success_move = True
                
                if success_move:
                    last_result_ms = (time.time() - reaction_start_time) * 1000
                    
                    # --- NEW: SAVE THE DATA ---
                    # Filter out "Accidental" glitches (super fast or super slow)
                    if 100 < last_result_ms < 1000:
                        trial_data.append({
                            "trial": len(trial_data) + 1,
                            "latency": last_result_ms,
                            "velocity": max_velocity
                        })
                    
                    state = "LANDED"
                    display_start = time.time()

            elif state == "LANDED":
                color = (0, 255, 0)
                if last_result_ms > 400: color = (0, 0, 255)
                text = f"{last_result_ms:.0f} ms"
                
                text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 2, 3)[0]
                text_x = (w - text_size[0]) // 2
                text_y = (h + text_size[1]) // 2
                cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)
                
                if time.time() - display_start > 1.5:
                    state = "WAIT"

        cv2.imshow(window_name, frame)
        if cv2.waitKey(5) & 0xFF == 27: break

cap.release()
cv2.destroyAllWindows()

# --- PHASE 3: THE REPORT (Matplotlib) ---
if len(trial_data) > 0:
    print("📊 Generating Scientific Report...")
    
    trials = [d['trial'] for d in trial_data]
    latencies = [d['latency'] for d in trial_data]
    velocities = [d['velocity'] for d in trial_data]

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    fig.suptitle('Oculomotor Biometrics: Latency & Velocity Analysis', fontsize=16)

    # Plot 1: Reaction Time
    bars = ax1.bar(trials, latencies, color='skyblue', edgecolor='black')
    ax1.axhline(y=200, color='r', linestyle='--', label='Healthy Human Baseline (200ms)')
    ax1.set_ylabel('Latency (ms)')
    ax1.set_title('Saccadic Reaction Time (Reflex Speed)')
    ax1.legend()
    ax1.bar_label(bars, fmt='%.0f')

    # Plot 2: Eye Speed
    ax2.plot(trials, velocities, marker='o', color='green', linewidth=2)
    ax2.set_xlabel('Trial Number')
    ax2.set_ylabel('Peak Velocity (deg/sec proxy)')
    ax2.set

✅ Experiment Started.
